In [13]:
import sys
import site
import os

# Add user site-packages to sys.path
user_site_packages = site.USER_SITE
if user_site_packages not in sys.path:
    sys.path.append(user_site_packages)

# Install packages
!{sys.executable} -m pip install langchain langchain_groq langchain_experimental langchain_ollama langchain_core pyiceberg pyarrow pyiceberg[glue] 

# Print paths to verify
print("\nPython paths:")
for p in sys.path:
    print(p)

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 14.8 MB/s eta 0:00:00

Python paths:
/
/emr/notebook-env/lib/python39.zip
/emr/notebook-env/lib/python3.9
/emr/notebook-env/lib/python3.9/lib-dynload

/emr/notebook-env/lib/python3.9/site-packages
/home/emr-notebook/.local/lib/python3.9/site-packages


In [20]:
import os
import boto3
from pyiceberg.catalog import load_catalog
import pandas as pd

# Get AWS credentials from the current session
session = boto3.Session(region_name='eu-central-1')
credentials = session.get_credentials()

# Set AWS environment variables
os.environ['AWS_ACCESS_KEY_ID'] = credentials.access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = credentials.secret_key
os.environ['AWS_SESSION_TOKEN'] = credentials.token
os.environ['AWS_DEFAULT_REGION'] = 'eu-central-1'

# Configure catalog
catalog = load_catalog(
    identifier='demo',
    type='glue',
    warehouse='s3://elasticmapreduce-uraeusdev/ICEBERG/',
    region='eu-central-1',
    glue_catalog_id=session.client('sts').get_caller_identity()['Account']
)

# Load table
table = catalog.load_table('uraeus_db.security_events')

# Convert to DataFrame directly
df = table.scan().to_arrow().to_pandas()

# View first few rows
print("First few rows:")
print(df.head())

First few rows:
                Alert_ID    ID                 Timestamp   VIN Severity  \
0  ALERT-20241019-INFO-1  SEV1 2024-10-19 13:05:48+00:00  VIN1     INFO   
1  ALERT-20241019-INFO-2  SEV2 2024-10-19 13:05:48+00:00  VIN2     INFO   
2  ALERT-20241019-INFO-3  SEV3 2024-10-19 13:05:48+00:00  VIN3     INFO   
3   ALERT-20241019-LOW-1  SEV1 2024-10-19 13:05:48+00:00  VIN1      LOW   
4   ALERT-20241019-LOW-2  SEV2 2024-10-19 13:05:48+00:00  VIN2      LOW   

                      SEV_Msg Origin NetworkType NetworkID Vehicle_Model  \
0  Test Security Event - INFO   TEST         CAN      NET1    TEST_MODEL   
1  Test Security Event - INFO   TEST         CAN      NET2    TEST_MODEL   
2  Test Security Event - INFO   TEST         CAN      NET3    TEST_MODEL   
3   Test Security Event - LOW   TEST         CAN      NET1    TEST_MODEL   
4   Test Security Event - LOW   TEST         CAN      NET2    TEST_MODEL   

   Fleet_Name  SEV_Name Rule_ID       Description SEV_Status  year  month  \